In [1]:
import pandas as pd
import numpy as np

In [2]:
customer_tb = pd.read_csv('./data/customer.csv', encoding='UTF-8')

In [7]:
reserve_tb = pd.read_csv('./data/reserve.csv', encoding='UTF-8')

In [14]:
production_tb = pd.read_csv('./data/production.csv', encoding='UTF-8')

In [21]:
production_missc_tb = pd.read_csv('./data/production_missing_category.csv', encoding='UTF-8')

## 9. 범주형
### 9.1. 범주형으로 변환
#### 범주형 변환

In [4]:
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

# 마스터 데이터가 생성됨
customer_tb['sex_c'] = pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

# 마스터 데이터가 생성되지 않음
# customer_tb['sex_c'] = customer_tb['sex'].astype('category')

customer_tb[['sex', 'sex_is_man', 'sex_c']]

,sex,sex_is_man,sex_c
0,man,True,man
1,man,True,man
2,woman,False,woman
3,man,True,man
4,man,True,man
...,...,...,...
995,man,True,man
996,man,True,man
997,woman,False,woman
998,woman,False,woman


In [5]:
customer_tb['sex_c'].cat.codes

0      0
1      0
2      1
3      0
4      0
      ..
995    0
996    0
997    1
998    1
999    0
Length: 1000, dtype: int8

In [6]:
customer_tb['sex_c'].cat.categories

Index(['man', 'woman'], dtype='object')

### 9.2. 더미 변수화
더미 변수화
- (범주형을 지원하지 않는 머신러닝 엔진을 위해) 범줏값을 플래그의 집합 값으로 변환하는 것
- 더미 변수를 범주 수보다 하나 줄여서 할 수도 있지만, 그러면 줄인 범주에 대해 가중치 등을 설정할 수 없다.

#### 더미 변수화

In [9]:
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])
dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)
dummy_vars

sex,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
995,1,0
996,1,0
997,0,1
998,0,1


### 9.3. 범주값의 집약
과학습과 지나치게 많은 범주값을 줄이기 위해, 범주값간에 집약을 하면 정확도가 높아질 수 있다.

#### 범주값의 잡약

In [11]:
customer_tb['age_rank'] = \
    pd.Categorical(np.floor(customer_tb['age']/10)*10)

customer_tb['age_rank'].cat.add_categories(['60+'], inplace=True)

customer_tb.loc[
    customer_tb['age_rank'].isin([60.0, 70.0, 80.0]),
    'age_rank'
] = '60+'

customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)

customer_tb['age_rank']

0      40
1      30
2      40
3      40
4      30
       ..
995    40
996    30
997    30
998    40
999    30
Name: age_rank, Length: 1000, dtype: category
Categories (5, object): [20, 30, 40, 50, 60+]

### 9.4. 범주값의 조합
데이터가 많아지기 때문에 데이터 양을 파악하면서 사용할 것
#### 범주값의 조합

In [12]:
customer_tb['sex_and_age'] = pd.Categorical(
    customer_tb[['sex', 'age']]
        .apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10), axis=1)
)
customer_tb['sex_and_age']

0        man_40.0
1        man_30.0
2      woman_40.0
3        man_40.0
4        man_30.0
          ...    
995      man_40.0
996      man_30.0
997    woman_30.0
998    woman_40.0
999      man_30.0
Name: sex_and_age, Length: 1000, dtype: category
Categories (14, object): [man_20.0, man_30.0, man_40.0, man_50.0, ..., woman_50.0, woman_60.0, woman_70.0, woman_80.0]

### 9.5. 범주형의 수치화
- 학습 데이터 양이 적을 때 고려됨
- 과학습이 발생하거나 원래 데이터의 의미를 잃을 수 있음 (그래서 기본적으로 추천되는 방법은 아님)

#### 범주형의 수치화

평균 불량률은 자신의 레코드는 제외한 평균으로 계산한다 (과학습 및 데이터 누수 방지)

In [16]:
fault_cnt_per_type = production_tb\
    .query('fault_flg') \
    .groupby('type')['fault_flg'] \
    .count()
fault_cnt_per_type

type
A    11
B     6
C    16
D     7
E    12
Name: fault_flg, dtype: int64

In [17]:
type_cnt = production_tb.groupby('type')['fault_flg'].count()
type_cnt

type
A    202
B    175
C    211
D    215
E    197
Name: fault_flg, dtype: int64

In [18]:
production_tb['type_fault_rate'] = production_tb[['type', 'fault_flg']] \
    .apply(
        lambda x: (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1),
        axis=1
    )

production_tb[['type', 'fault_flg', 'type_fault_rate']]

,type,fault_flg,type_fault_rate
0,E,False,0.061224
1,D,False,0.032710
2,E,False,0.061224
3,B,False,0.034483
4,B,False,0.034483
...,...,...,...
995,C,False,0.076190
996,D,False,0.032710
997,B,False,0.034483
998,D,False,0.032710


### 9.6. 범주형의 보완
결손에 대한 보완 방법
- 고정값으로 보완  
결손을 의미하는 고정값을 만들어서 보완. 존재하지 않는 범주가 늘어나므로 비추천
- 집계값으로 보완  
최빈값을 계산해 보완값으로 사용. 지정한 최빈값 데이터가 늘어나므로 결손값이 많을 때는 비추천
- 결손이 발생하지 않은 데이터에 기반한 예측값으로 보완  
- 시간 관계로 보완
- 다중대입법으로 보완  
보완한 데이터셋을 여러 개 만들어 각 데이터셋을 해석
- 최대 가능도로 보완  
잠재 변수를 도입해 EM 알고리즘을 사용, 가능도를 최대화하여 결손값을 보완

#### KNN을 이용한 보완
결손이 발생하지 않은 데이터를 이용한 예측 결과에서 결손값을 보완

In [23]:
from sklearn.neighbors import KNeighborsClassifier

production_missc_tb.replace('None', np.nan, inplace=True)

train = production_missc_tb.dropna(subset=['type'], inplace=False)

test = production_missc_tb \
    .loc[production_missc_tb.index.difference(train.index), :]

kn = KNeighborsClassifier(n_neighbors=3)

kn.fit(train[['length', 'thickness']], train['type'])

test['type'] = kn.predict(test[['length', 'thickness']])
test['type']

8      E
26     E
30     E
36     A
41     E
      ..
971    A
980    E
983    B
992    A
996    A
Name: type, Length: 100, dtype: object